# This version uses the same logic as in v_10 but uses one week as its timeframe

#### Have also updated the FCR and aFRR functions to make them faster and make FCR handle missing values

In [ ]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, analysis, timeframes, data_handling
import numpy as np
import pickle
from datetime import datetime



In [ ]:
timeframe = timeframes.one_week
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)

In [ ]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [ ]:

markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())
price_list = [market.price_data for market in M]
volume_list = [market.volume_data for market in M]


In [ ]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

In [ ]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
total_response_time = np.sum(R_h_l) # total response time
#total_flex = total_up_flex + total_down_flex
average_response_time = total_response_time/ (len(H)*len(L))
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex} MW")
print(f"Total down flex volume: {total_down_flex} MW")
print(f"Average flex volume pr hour up: {hourly_flex_up} MWh")
print(f"Average flex volume pr hour down: {hourly_flex_down} MWh")
print(f"Average response time: {average_response_time} seconds")

In [ ]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]

In [ ]:
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H = H, M = M, L = L, freq_data= freq_data, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, P_h_m = P_h_m, Vp_h_m = Vp_h_m, F = F, markets_dict= markets_name_dict, timeframe = timeframe)


In [ ]:
compatible_list = utils.get_compatibility_dict(L, M)

In [ ]:
import math
def get_batched_versions(L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], H : [pd.Timestamp], F : dict, freq_data :pd.DataFrame, P_h_m : np.array, Vp_h_m : np.array, R_h_l : np.array, Fu_h_l : np.array, Fd_h_l : np.array):
    """ Function to only fetch the collections that are batched in the batched optimization model. Can use this to check if there is something wrong with some of the batches.

    Args:
        L (list(new_meters.PowerMeter]): set of all meters
        M (list(final_markets.ReserveMarket]): set of all markets
        H (list(pd.Timestamp]): set of all hours
        F (dict): Dictionary to find the activation percentages for each market and hour
        freq_data (pd.DataFrame): dataframe with the frequency data
        P_h_m (np.array): The price for each hour and market
        Vp_h_m (np.array): The volume for each hour and market
        R_h_l (np.array): Response time for each load each hour
        Fu_h_l (np.array): Up flex volume for each load that are compatible with up markets for each hour 
        Fd_h_l (np.array): Down flex volume for each load that are compatible with down markets for each hour
       

    Returns:
        batched_results (dict): Dictionary with the batched results
    """
    batch_size = 24  # For example, batching by 24 hours
    num_batches = math.ceil(len(H) / batch_size)
    aggregated_results = {
        'batched H': [],
        'batched R_h_l': [],
        'batched Fu_h_l': [],
        'batched Fd_h_l': [],
        'batched Vp_h_m': [],
        'batched P_h_m': [],
        'batched timeframes': [],
        'batched Ir_h_l_m': [],
        'batched Ia_h_l_m': [],
        'batched Va_hm': []
    }
    market_name_dict = {m.name : m for m in M}

    for b in range(num_batches):
        # Determine the subset of hours for this batch
        start_index = b * batch_size
        end_index = min((b + 1) * batch_size, len(H))
        batch_H = H[start_index:end_index]

        # Slice numpy arrays for the current batch
        batch_R_h_l = R_h_l[start_index:end_index, :]
        batch_Fu_h_l = Fu_h_l[start_index:end_index, :]
        batch_Fd_h_l = Fd_h_l[start_index:end_index, :]
        batch_Vp_h_m = Vp_h_m[start_index:end_index, :]
        batch_P_h_m = P_h_m[start_index:end_index, :]
        tf = timeframes.TimeFrame(year = 2023, start_month = 6, end_month = 6, start_day = batch_H[0].day, end_day = batch_H[0].day, start_hour = 0, end_hour = 23)

        # the income
        batch_Ir_hlm, batch_Ia_hlm, batch_Va_hm = utils.get_income_dictionaries(H = batch_H, M = M, L =L, freq_data = freq_data, Fu_h_l = batch_Fu_h_l, Fd_h_l = batch_Fd_h_l, P_h_m = batch_P_h_m, Vp_h_m = batch_Vp_h_m, F =F, markets_dict = market_name_dict, timeframe = tf)
       

        # Run the optimization model for this batch
        #_, x, y, w, d = run_optimization_model(L= L, M= M, H = batch_H,F= F, Ir_hlm= batch_Ir_hlm, Ia_hlm= batch_Ia_hlm, Va_hm= batch_Va_hm, Vp_h_m= batch_Vp_h_m, Vm_m=Vm_m, R_m=R_m, R_h_l=batch_R_h_l, Fu_h_l=batch_Fu_h_l, Fd_h_l=batch_Fd_h_l, compatible_list=compatible_list, log_filename=log_filename, model_name=f"{model_name}_batch_{b}")
        # Store results
        #aggregated_results['models'].append(model)
        aggregated_results['batched H'].append(batch_H)
        aggregated_results['batched R_h_l'].append(batch_R_h_l)
        aggregated_results['batched Fu_h_l'].append(batch_Fu_h_l)
        aggregated_results['batched Fd_h_l'].append(batch_Fd_h_l)
        aggregated_results['batched Vp_h_m'].append(batch_Vp_h_m)
        aggregated_results['batched P_h_m'].append(batch_P_h_m)
        aggregated_results['batched timeframes'].append(tf)
        aggregated_results['batched Ir_h_l_m'].append(batch_Ir_hlm)
        aggregated_results['batched Ia_h_l_m'].append(batch_Ia_hlm)
        aggregated_results['batched Va_hm'].append(batch_Va_hm)
        


    # Process aggregated_results as needed
    return aggregated_results

In [ ]:
batch_vals = get_batched_versions(L= L, M= M, H = H, F = F,freq_data=freq_data, P_h_m=P_h_m , Vp_h_m =Vp_h_m, R_h_l = R_h_l, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l)

In [ ]:
#for day in range(len(batch_vals["batched H"])):

"""
'batched H': [],
'batched R_h_l': [],
'batched Fu_h_l': [],
'batched Fd_h_l': [],
'batched Vp_h_m': [],
'batched P_h_m': [],
'batched timeframes': [],
'batched Ir_h_l_m': [],
'batched Ia_h_l_m': [],
'batched Va_hm': []
"""


for h in range(len(batch_vals["batched H"][6])):
    for l in range(len(L)):
        """if np.isnan(batch_vals["batched R_h_l"][6][h,l]):
                print(f" R_h_l is nan for Hour: {h}, Meter: {l}")
        if np.isnan(batch_vals["batched Fu_h_l"][6][h,l]):
            print(f" Fu_h_l is nan for Hour: {h}, Meter: {l}")
        if np.isnan(batch_vals["batched Fd_h_l"][6][h,l]):
            print(f"Fd_h_l is nan for Hour: {h}, Meter: {l}")"""
        for m in range(12,22):
            """if np.isnan(batch_vals["batched Vp_h_m"][6][h,m]):
                print(f" Vp_h_m is nan for Hour: {h}, Meter: {l}, Market: {m}")
            if np.isnan(batch_vals["batched P_h_m"][6][h,m]):
                print(f" P_h_m is nan for Hour: {h}, Meter: {l}, Market: {m}")
            if np.isnan(batch_vals["batched Va_hm"][6][h,m]):
                print(f" Va_hm is nan for Hour: {h}, Meter: {l}, Market: {m}")"""
                
        
            if np.isnan(batch_vals["batched Ia_h_l_m"][6][h,l,m]):
                print(f" Ia_hlm is nan for Hour: {h}, Meter: {l}, Market: {m}")
            if np.isinf(batch_vals["batched Ia_h_l_m"][6][h,l,m]):
                print(f" Ia_hlm is inf for Hour: {h}, Meter: {l}, Market: {m}")
            if np.isnan(batch_vals["batched Ir_h_l_m"][6][h,l,m]):
                print(f" Ir_hlm is nan for Hour: {h}, Meter: {l}, Market: {m}")
            if np.isinf(batch_vals["batched Ir_h_l_m"][6][h,l,m]):
                print(f" Ir_hlm is inf for Hour: {h}, Meter: {l}, Market: {m}")
                
    

### There is only problems with the Ia_hlm and none of the other collections. Ia_hlm has nan values for markets 12-21 which is the fcr_n markets. I have to figure out why this problem is occuring.

In [ ]:
aggregated_results  = utils.run_batched_optimization_model(L= L , M = M, H = H, F = F,freq_data=freq_data, P_h_m=P_h_m , Vp_h_m =Vp_h_m, Vm_m = Vm_m, R_m = R_m, R_h_l = R_h_l, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, compatible_list = compatible_list, log_filename="batched_week_v4.log", model_name= "weekly_model_batched")

In [ ]:
def dissaggregate_results(aggregated_results : dict):
    daily_vals = []
    x_vals = aggregated_results['x_values']
    y_vals = aggregated_results['y_values']
    w_vals = aggregated_results['w_values']
    for day_nr in range(len(x_vals)):
        daily_vals.append({"x_values": x_vals[day_nr], "y_values": y_vals[day_nr], "w_values": w_vals[day_nr]})
    return daily_vals

In [ ]:
daily_vals_list = dissaggregate_results(aggregated_results)

In [ ]:
def test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions, F):
    """ function to test the validity of the solution provided by a solver

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        y (dict): dictionary of the binary variable which tells if a market has any bids
        w (dict): dictionary of the binary variable which tells if a market is activated
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour
        F (pd.DataFrame): dictionary for frequency data
    Returns:
        str : a string that tells if the solution is valid. If not valid, the function will raise an error
    """
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert round(sum(x[h, l, m].X for m in range(len(M))), 5) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                x_val= round(x[h, l, m].X, 5)
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x_val== 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x_val == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                #elif market.direction == "both" and load.direction != "both":
                    #assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"
                elif market.area != load.area:
                    assert x_val == 0, f"Asset {l} in area {load.area} connected to market {m} in area {market.area} at hour {h}"
                
                # Response time constraints
                assert x_val * load.response_time <= market.response_time * round(y[h, m].X, 5), f"Asset {l} connected to market {m} at hour {h} violates response time constraint"
                
        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert round(sum(x[h, l, m].X for l in range(len(L))), 5) <= len(L) * round(y[h, m].X, 5), f"More than allowed assets connected to market {m} at hour {h} to market {m}"

            #total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))

            # Min volume constraint
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else: # direction = "both"
                if dominant_directions[h] == "up":
                    total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            
            assert round(total_flex_volume, 5) >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"
            
            # Max volume constraint for both capacity and activation
            if market.direction == "up":
                total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")

            elif market.direction == "down":
                total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")

            else:
                """if dominant_directions[h] == "up":
                    total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")"""
                total_up_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                total_down_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
                up_frac, down_frac = F[h,m]
                total_max_volume = (total_up_max_volume * up_frac + total_down_max_volume * down_frac)
            
             # Assert the constraints
            activation_constraint = round(total_max_volume, 5)  * round(w[h,m].X, 5) <= Va_hm[h,m]
            assert activation_constraint, f"Activation constraint violated for hour {h}, market {m}"
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * round(y[h,m].X, 5), f"Maximum volume constraint violated at hour {h} for market {m}"
    return "Solution is valid"

In [ ]:
H_batches = [H[:24], H[24:]]
for index, daily_val in enumerate(daily_vals_list):
    utils.test_solution_validity(daily_val["x_values"], daily_val["y_values"], daily_val["w_values"], Va_hm, L, M, H_batches[index], dominant_directions= dominant_directions, F = F)
    print(f" Solution for day {daily_vals_list.index(daily_val)} is valid")
    # must split the H list into days

### possible reasons for failure of the second day batched version:
# the indexes becomes wrong when the data is batched


In [ ]:
afrr_activation_up = utils.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Up") #  a dataframe of the activation volumes for afrr up for each hour in the timeframe
afrr_activation_down = utils.get_afrr_activation_data(tf = timeframe, afrr_directory = '../master-data/aFRR_activation/', direction = "Down") #  a dataframe of the activation volumes for afrr down for each hour in the timeframe
frequency_quarter_dict = utils.find_frequency_quarters(freq_df = freq_data, hours = H, index = True) # a dictionary of the frequency quarters for each hour

In [ ]:
afrr_activation_up.isna()

In [ ]:
#sum(x[7, l, 21].X for l in range(len(L)))

In [ ]:
#len(L) * y[7, 21].X

In [ ]:
#round(sum(x[7, 839, m].X for m in range(len(M))), 6)

In [ ]:
import pickle

In [ ]:
# Extract binary variable values from the original model
#new_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}


In [ ]:
"""# Load the saved values
with open('current_x_values_for_week.pkl', 'rb') as f:
    original_x_values = pickle.load(f)

old_dict = utils.get_market_count_dict(original_x_values)
new_dict = utils.get_market_count_dict(new_x_values)

differences = {}
for key in old_dict:
    if not old_dict[key].equals(new_dict[key]):
        differences[key] = (new_dict[key], old_dict[key])
        

for key, (orig_val, mod_val) in differences.items():
    print(f"Difference for hour {key}: \n Original={display(orig_val)}, \n  Modified={display(mod_val)}")"""


In [ ]:
"""# Extract binary variable values from the original model
current_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(current_x_values, f)"""
                